In [3]:
%pip install docker pymongo pandas

You should consider upgrading via the '/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Setup mongo and import data

In [4]:
import os
import docker

docker_client = docker.from_env()
MONGO_PORT = 27018

mongo_container = docker_client.containers.run(
    'mongo:latest',
    detach=True,
    name='mongo-data-prep',
    remove=True,
    ports={'27017/tcp': MONGO_PORT},
    mem_limit='2G',
    volumes={
        os.path.join(os.getcwd(), 'db'): {'bind': '/data/db', 'mode': 'rw'},
    }
)

In [5]:
from pymongo import MongoClient
mongo_client = MongoClient('127.0.0.1', MONGO_PORT)
db = mongo_client['data-prep']

In [6]:
sales_small_col = db.get_collection('sales-small')
sales_small_col.drop()

In [7]:
from pathlib import Path
import json
sales_small_data = json.loads(Path('/home/david/dev/toucan-toco/python-data-prep/data/sales-small.json').read_text())
sales_small_col.insert_many(sales_small_data['data'])

In [8]:
import pandas as pd

1. Simulating a simple weaverbird pipeline with just a filter, on a very small domain:

In [55]:
'''
{
    "domain": "sales-small",
    "name": "domain"
},
'''
def get_domain(domain):
    return pd.DataFrame(db.get_collection(domain).find({}, {'_id': False}))
df = get_domain('sales-small')

In [10]:
'''
{
    "name": "filter",
    "condition": {
      "column": "Payment_Type",
      "operator": "eq",
      "value": "Mastercard"
    }
}
'''
def filter(df, column, value):
    return df[df[column] == value]
df1 = filter(df, 'Payment_Type', 'Mastercard')

In [11]:
%%time

'''
Whole pipeline
'''

df = get_domain('sales-small')
filter(df, 'Payment_Type', 'Mastercard')

CPU times: user 7.71 ms, sys: 0 ns, total: 7.71 ms
Wall time: 7.8 ms


,_id,Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude
1,5f9142baee2fc549d19201bd,1/5/09 4:10,Product1,1200,Mastercard,Nicola,Roodepoort,Gauteng,South Africa,1/5/09 2:33,1/7/09 5:13,-26.166667,27.866667
2,5f9142baee2fc549d19201be,1/4/09 13:17,Product1,1200,Mastercard,Renee Elisabeth,Tel Aviv,Tel Aviv,Israel,1/4/09 13:03,1/4/09 22:10,32.066667,34.766667
3,5f9142baee2fc549d19201bf,1/2/09 6:17,Product1,1200,Mastercard,carolina,Basildon,England,United Kingdom,1/2/09 6:00,1/2/09 6:08,51.500000,-1.116667
6,5f9142baee2fc549d19201c2,1/5/09 8:58,Product2,3600,Mastercard,Marcia,Telgte,Nordrhein-Westfalen,Germany,9/1/08 3:39,1/14/09 2:07,52.333333,7.900000
9,5f9142baee2fc549d19201c5,1/15/09 12:54,Product1,1200,Mastercard,Annelies,Ile-Perrot,Quebec,Canada,1/15/09 12:22,1/16/09 7:53,45.400000,-73.933333
15,5f9142baee2fc549d19201cb,1/12/09 15:12,Product1,1200,Mastercard,David,Deptford,NJ,United States,1/12/09 14:07,1/19/09 3:47,39.838060,-75.153060
16,5f9142baee2fc549d19201cc,1/19/09 16:10,Product1,1200,Mastercard,Frank,Old Greenwich,CT,United States,1/19/09 15:31,1/19/09 16:00,41.022780,-73.565280
17,5f9142baee2fc549d19201cd,1/20/09 6:03,Product1,1200,Mastercard,Andrea,Shreveport,LA,United States,1/20/09 5:13,1/20/09 7:15,32.525000,-93.750000
26,5f9142baee2fc549d19201d6,1/4/09 9:54,Product1,1200,Mastercard,AMY,The Woodlands,TX,United States,12/30/08 20:41,1/25/09 18:23,30.157780,-95.489170
27,5f9142baee2fc549d19201d7,1/22/09 15:32,Product1,1200,Mastercard,Tara,Killiney,Dublin,Ireland,2/27/07 11:35,1/26/09 4:32,53.252222,-6.112500


2. Simulating a groupby, keeping the original format, on larger domain

In [12]:
# Inserting data
sales_col = db.get_collection('sales')
sales_col.drop()
data = pd.read_csv('/home/david/dev/toucan-toco/python-data-prep/data/sales.csv').to_dict(orient='rows')
sales_col.insert_many(data)

/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


## Aggregate

```json
{
    "name": "aggregate",
    "on": [
      "DATE"
    ],
    "aggregations": [
      {
        "columns": [
          "WEEKLY_SALES"
        ],
        "newcolumns": [
          "Transaction_date-sum"
        ],
        "aggfunction": "sum"
      }
    ],
    "keepOriginalGranularity": true
}
```

In [20]:
%%time

%time df = get_domain('sales')
%time df['WEEKLY_SALES-sum'] = df.groupby(by=['DATE'])['WEEKLY_SALES'].transform('sum')
df

CPU times: user 2.76 s, sys: 47.7 ms, total: 2.81 s
Wall time: 3.03 s
CPU times: user 80.2 ms, sys: 0 ns, total: 80.2 ms
Wall time: 81.9 ms
CPU times: user 2.84 s, sys: 47.8 ms, total: 2.89 s
Wall time: 3.11 s


,_id,STORE_ID,DEPT_ID,DATE,WEEKLY_SALES,WEEKLY_SALES-sum
0,5f9142bdee2fc549d19201ee,1,1,05/02/2010,24924.50,49750740.50
1,5f9142bdee2fc549d19201ef,1,1,12/02/2010,46039.49,48336677.63
2,5f9142bdee2fc549d19201f0,1,1,19/02/2010,41595.55,48276993.78
3,5f9142bdee2fc549d19201f1,1,1,26/02/2010,19403.54,43968571.13
4,5f9142bdee2fc549d19201f2,1,1,05/03/2010,21827.90,46871470.30
...,...,...,...,...,...,...
421565,5f9142c0ee2fc549d19870ab,45,98,28/09/2012,508.37,43734899.40
421566,5f9142c0ee2fc549d19870ac,45,98,05/10/2012,628.10,47566639.31
421567,5f9142c0ee2fc549d19870ad,45,98,12/10/2012,1061.02,46128514.25
421568,5f9142c0ee2fc549d19870ae,45,98,19/10/2012,760.01,45122410.57


In [19]:
%%time
'''Compare with mongo'''

sales_agg = [
    {
        "$group": {
            "_id": {
                "DATE": "$DATE"
            },
            "_vqbDocsArray": {
                "$push": "$$ROOT"
            },
            "WEEKLY_SALES-sum": {
                "$sum": "$WEEKLY_SALES"
            }
        }
    },
    {
        "$unwind": "$_vqbDocsArray"
    },
    {
        "$replaceRoot": {
            "newRoot": {
                "$mergeObjects": [
                    "$_vqbDocsArray",
                    "$$ROOT"
                ]
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "_vqbDocsArray": 0
        }
    }
]

# Memory fail without allowDiskUse
sales_col.aggregate(sales_agg)

OperationFailure: Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in., full error: {'ok': 0.0, 'errmsg': "Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in.", 'code': 292, 'codeName': 'QueryExceededMemoryLimitNoDiskUseAllowed'}

In [30]:
%%time
%time res = sales_col.aggregate(sales_agg, allowDiskUse=True)
%time list(res)

CPU times: user 2.99 ms, sys: 0 ns, total: 2.99 ms
Wall time: 1.57 s
CPU times: user 1.36 s, sys: 83.9 ms, total: 1.45 s
Wall time: 3.21 s
CPU times: user 1.37 s, sys: 83.9 ms, total: 1.45 s
Wall time: 4.79 s


[{'STORE_ID': 1,
  'DEPT_ID': 1,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 20398.09,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 2,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 46991.58,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 3,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 8734.19,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 4,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 34451.9,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 5,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 23598.55,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 6,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 3249.27,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 7,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 20144.71,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 8,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 35319.05,
  'WEEKLY_SALES-sum': 

Result:
- fail without allowDiskUse
- double time in python due to the extraction from mongo of the whole dataset (3s)
- the operation itself is notably faster in Python (80ms vs 1.5s)

=> we will need to find a smart way to cache the extraction, maybe in a parquet file or something else fast

### Sort
```json
{
    "name": "sort",
    "columns": [
      {
        "column": "DATE",
        "order": "asc"
      }
    ]
}
```


In [21]:
%%time

def sort(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    return df.sort_values([c['column'] for c in columns], ascending=[(c.get('order') == 'desc') for c in columns])

%time df = get_domain('sales')
%time df1 = sort(df, [{'column': 'DATE', 'order': 'asc'}])
df1

CPU times: user 2.98 s, sys: 55.5 ms, total: 3.03 s
Wall time: 3.22 s
CPU times: user 886 ms, sys: 11.6 ms, total: 897 ms
Wall time: 892 ms
CPU times: user 3.86 s, sys: 67.1 ms, total: 3.93 s
Wall time: 4.11 s


,_id,STORE_ID,DEPT_ID,DATE,WEEKLY_SALES
404092,5f9142bfee2fc549d1982c6a,43,94,31/12/2010,41000.74
91966,5f9142bdee2fc549d193692c,10,33,31/12/2010,9978.04
341181,5f9142bfee2fc549d19736ab,36,8,31/12/2010,3362.24
416393,5f9142bfee2fc549d1985c77,45,33,31/12/2010,2924.12
123735,5f9142beee2fc549d193e545,13,48,31/12/2010,1309.00
...,...,...,...,...,...
286751,5f9142bfee2fc549d196620d,30,2,01/04/2011,13535.83
65225,5f9142bdee2fc549d19300b7,7,54,01/04/2011,28.00
184906,5f9142beee2fc549d194d438,19,74,01/04/2011,16831.26
392696,5f9142bfee2fc549d197ffe6,42,13,01/04/2011,14201.53


In [22]:
%%time
sales_sort = {
    "$sort": {
      "Account_Created": 1
    }
}
sales_col.aggregate(sales_agg)

OperationFailure: Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in., full error: {'ok': 0.0, 'errmsg': "Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in.", 'code': 292, 'codeName': 'QueryExceededMemoryLimitNoDiskUseAllowed'}

In [29]:
%%time
%time res = sales_col.aggregate(sales_agg, allowDiskUse=True)
%time list(res)

CPU times: user 3.03 ms, sys: 0 ns, total: 3.03 ms
Wall time: 1.53 s
CPU times: user 1.43 s, sys: 141 ms, total: 1.57 s
Wall time: 3.33 s
CPU times: user 1.43 s, sys: 141 ms, total: 1.57 s
Wall time: 4.86 s


[{'STORE_ID': 1,
  'DEPT_ID': 1,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 20398.09,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 2,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 46991.58,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 3,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 8734.19,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 4,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 34451.9,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 5,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 23598.55,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 6,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 3249.27,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 7,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 20144.71,
  'WEEKLY_SALES-sum': 43458991.190000005},
 {'STORE_ID': 1,
  'DEPT_ID': 8,
  'DATE': '01/04/2011',
  'WEEKLY_SALES': 35319.05,
  'WEEKLY_SALES-sum': 

Same conclusions:
- mongo needs allowDiskUse
- python operation notably faster than mongo (800ms vs 1.34s)

## Join

```json
{
    "name": "join",
    "right_pipeline": [
      {
        "domain": "markdowns",
        "name": "domain"
      }
    ],
    "type": "left",
    "on": [
      [
        "$STORE_ID",
        "$STORE_ID"
      ],
      [
        "DATE",
        "DATE"
      ]
    ]
}
```

In [25]:
# Inserting data
markdowns_col = db.get_collection('markdowns')
markdowns_col.drop()
data = pd.read_csv('/home/david/dev/toucan-toco/python-data-prep/data/markdowns.csv').to_dict(orient='rows')
markdowns_col.insert_many(data)

/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [34]:
%%time

def join(left:pd.DataFrame, right: pd.DataFrame, type: str, on: list):
    return pd.merge(left, right, how=type, left_on=[o[0] for o in on], right_on=[o[1] for o in on])
%time df_sales = get_domain('sales')
%time df_markdowns = get_domain('markdowns')
%time df = join(df_sales, df_markdowns, type='left', on=[['STORE_ID', 'STORE_ID'], ['DATE', 'DATE']])
df

CPU times: user 2.97 s, sys: 47.9 ms, total: 3.01 s
Wall time: 3.24 s
CPU times: user 30.4 ms, sys: 71 µs, total: 30.4 ms
Wall time: 32.7 ms
CPU times: user 161 ms, sys: 4.01 ms, total: 165 ms
Wall time: 164 ms
CPU times: user 3.16 s, sys: 52 ms, total: 3.21 s
Wall time: 3.44 s


,_id_x,STORE_ID,DEPT_ID,DATE,WEEKLY_SALES,_id_y,MARKDOWN_1,MARKDOWN_2,MARKDOWN_3,MARKDOWN_4,MARKDOWN_5
0,5f9142bdee2fc549d19201ee,1,1,05/02/2010,24924.50,NaN,NaN,NaN,NaN,NaN,NaN
1,5f9142bdee2fc549d19201ef,1,1,12/02/2010,46039.49,NaN,NaN,NaN,NaN,NaN,NaN
2,5f9142bdee2fc549d19201f0,1,1,19/02/2010,41595.55,NaN,NaN,NaN,NaN,NaN,NaN
3,5f9142bdee2fc549d19201f1,1,1,26/02/2010,19403.54,NaN,NaN,NaN,NaN,NaN,NaN
4,5f9142bdee2fc549d19201f2,1,1,05/03/2010,21827.90,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
421565,5f9142c0ee2fc549d19870ab,45,98,28/09/2012,508.37,5f9149afee2fc549d1988056,4556.61,20.64,1.50,1601.01,3288.25
421566,5f9142c0ee2fc549d19870ac,45,98,05/10/2012,628.10,5f9149afee2fc549d1988057,5046.74,NaN,18.82,2253.43,2340.01
421567,5f9142c0ee2fc549d19870ad,45,98,12/10/2012,1061.02,5f9149afee2fc549d1988058,1956.28,NaN,7.89,599.32,3990.54
421568,5f9142c0ee2fc549d19870ae,45,98,19/10/2012,760.01,5f9149afee2fc549d1988059,2004.02,NaN,3.18,437.73,1537.49


In [28]:
%%time

join_with_markdowns_agg = [
  {
    "$lookup": {
      "from": "markdowns",
      "let": {
        "vqb_STORE_ID": "$STORE_ID",
        "vqb_DATE": "$DATE"
      },
      "pipeline": [
        {
          "$match": {
            "$expr": {
              "$and": [
                {
                  "$eq": [
                    "$STORE_ID",
                    "$$vqb_STORE_ID"
                  ]
                },
                {
                  "$eq": [
                    "$DATE",
                    "$$vqb_DATE"
                  ]
                }
              ]
            }
          }
        }
      ],
      "as": "_vqbJoinKey"
    }
  },
  {
    "$unwind": {
      "path": "$_vqbJoinKey",
      "preserveNullAndEmptyArrays": True
    }
  },
  {
    "$replaceRoot": {
      "newRoot": {
        "$mergeObjects": [
          "$_vqbJoinKey",
          "$$ROOT"
        ]
      }
    }
  },
  {
    "$project": {
      "_vqbJoinKey": 0,
      "_id": 0
    }
  }
]
%time res = sales_col.aggregate(join_with_markdowns_agg)
%time list(res)

CPU times: user 2.25 ms, sys: 0 ns, total: 2.25 ms
Wall time: 272 ms
CPU times: user 705 ms, sys: 51.2 ms, total: 756 ms
Wall time: 8min 49s


OperationFailure: Error in $cursor stage :: caused by :: operation was interrupted, full error: {'ok': 0.0, 'errmsg': 'Error in $cursor stage :: caused by :: operation was interrupted', 'code': 11601, 'codeName': 'Interrupted'}

## Synthèse des résultats

### Temps total
Incluant l'extraction des données vers le processus du notebook

| operation | temps total mongo | temps total python |
|---|---|---|
| aggregate | 4.79s | 3.11s |
| sort | 4.86s | 4.11s |
| join | killed après 8min | 3.44s |

### Temps de l'opération uniquement

| operation | temps mongo | temps python |
|---|---|---|
| aggregate | 3.21 s | 81.9 ms |
| sort | 3.33 s | 892 ms |
| join | killed après 8min | 164ms |


## Mitiger le temps initial de load de la table

1. Sampling
2. Cache

Pour 2, est-ce qu'il serait plus rapide de dumper le résulat la première fois dans un format sur le disque dans un format très rapide à charger?

In [56]:
df = get_domain('sales')

### Synthese

| format | read | write |
|--------|------|-------|
| pickle | 158 ms | 64.8 ms |
| hdf5 | 846 ms | 154 ms |
| feather | 14.3 ms | 52.2 ms|
| parquet (pyarrow) | 25.6 ms | 90.1 ms |
| parquet (fastparquet) | 73.7 ms | 217 ms |

/!\ il faut bien enlever la colonne ObjectId, ca change bcp les résultats!
Il y a des options possible de compression pour certains formats, que je n'ai pas explorées ici.

Ressource comparable trouvée sur les internets: https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [58]:
%timeit df.to_pickle('sales.pkl')
%timeit pd.read_pickle('sales.pkl')

158 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
64.8 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%pip install tables

  Using cached tables-3.6.1-cp38-cp38-manylinux1_x86_64.whl (4.3 MB)
  Using cached numexpr-2.7.1-cp38-cp38-manylinux1_x86_64.whl (164 kB)
You should consider upgrading via the '/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [59]:
with pd.HDFStore('store.h5') as store:
    %timeit store['sales'] = df

with pd.HDFStore('store.h5') as store:
    %timeit store.get('sales')

154 ms ± 6.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
846 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%pip install pyarrow fastparquet

  Using cached fastparquet-0.4.1.tar.gz (28.6 MB)
     |████████████████████████████████| 3.1 MB 458 kB/s eta 0:00:01
  Using cached thrift-0.13.0.tar.gz (59 kB)
  Using cached llvmlite-0.34.0-cp38-cp38-manylinux2010_x86_64.whl (24.6 MB)
Using legacy 'setup.py install' for fastparquet, since package 'wheel' is not installed.
Using legacy 'setup.py install' for thrift, since package 'wheel' is not installed.
    Running setup.py install for thrift ... - \ | / - \ | done
    Running setup.py install for fastparquet ... - \ | / - done
You should consider upgrading via the '/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [65]:
# Warning: ObjectId cannot be serialized in Feather format
%timeit df.to_feather('sales.feather')
%timeit pd.read_feather('sales.feather')


50.9 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
15.1 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
%timeit df.to_parquet('sales.parquet', engine='pyarrow', compression=None)
%timeit pd.read_parquet('sales.parquet', engine='pyarrow')

%timeit df.to_parquet('sales.parquet', engine='fastparquet', compression=None)
%timeit pd.read_parquet('sales.parquet', engine='fastparquet')

90.1 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
25.6 ms ± 434 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
217 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
73.7 ms ± 532 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
